# Part 2

- Below is the code for requide task, of creating voice bot with response.
- I have created voice ivr, for Ace hotels which handles booking and other stuff related to room management of the hotel

- pre requisites
- Need ollama mistral model installed - "ollama run mistral" in background
- ffmpg library required to be installed
- Required libraries are imported as below.

In [4]:
import sounddevice as sd
import librosa
from transformers import pipeline
import whisper
import ollama
import numpy as np
import wavio

# Play the greeting audio
def play_audio(data, fs):
    sd.play(data, fs)
    sd.wait()
    
def play_audio_file(file_path):
    # Read the audio file
    wav_obj = wavio.read(file_path)
    
    # Get data and sampling rate from the Wav object
    wav_data = wav_obj.data
    fs = wav_obj.rate

    # Play the audio
    sd.play(wav_data, fs)
    sd.wait()
    
# Step 2: Record live audio using sounddevice
def record_audio(file_path, duration=6, fs=44100):
    print("\nRecording...")
    audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()
    print("\nAudio Captured")
    
    # Save the audio data as WAV file using wavio
    audio_data = audio_data.flatten()
    wavio.write(file_path, audio_data, fs, sampwidth=2)
    
    return audio_data

# Transcribe audio using OpenAI's Whisper model
def transcribe_audio_whisper(audio_data):
    transcriber = pipeline(model="openai/whisper-medium", task="automatic-speech-recognition")
    transcription = transcriber(audio_data)
    print("Transcription:", transcription['text'])
    return transcription['text']

# Use local language model with ollama for intent detection
def generate_text_with_ollama(prompt):
    stream = ollama.chat(
        model='mistral',
        messages=[{'role': 'user', 'content': "tell me the intent in one word is booking or cancelling or other in this text -" + prompt}],
        stream=True)
    text_list = []
    for chunk in stream:
        content = chunk['message']['content']
        
        text_list.append(content)
    
    if "book" or "booking" in text_list:
        return 1
    elif "cancel" or "cancelling" or "elling" in text_list:
        return 2
    else:
        return 3

# response basing on intent
def identify_intent(intent_status):
    if intent_status == 1:
        play_audio_file(response1)
        record_audio(recorded_audio_path)
        play_audio_file(final_response)
    elif intent_status == 2:
        play_audio_file(response2)
        record_audio(recorded_audio_path)
        play_audio_file(final_response)
    else:
        play_audio_file(response3)

# Sentiment analysis (using Huggingface sentiment-analysis model)
def sentiment_analysis(text):
    sentiment_analyzer = pipeline('sentiment-analysis')
    result = sentiment_analyzer(text)
    print("Sentiment:", result[0]['label'])

# Example usage:
recorded_audio_path = "recorded_audio.wav"
greeting_audio_path = "Hello Welcome to Ace.wav"
response1 = "response1.wav"
response2 = "response2.wav"
response3 = "response3.wav"
final_response = "finalresponse.wav"
fs = 44100

# start of the code
play_audio_file(greeting_audio_path)

recorded_audio_data = record_audio(recorded_audio_path)

recorded_text = transcribe_audio_whisper(recorded_audio_path)
sentiment_analysis(recorded_text)

prompt_ollama = recorded_text
status = generate_text_with_ollama(prompt_ollama)

identified_intent = identify_intent(status)


Recording...

Audio Captured


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Transcription:  I want to cancel my booking.
Sentiment: NEGATIVE

Recording...

Audio Captured
